In [1]:
from time import time
from math import log
import numba as nb
import numpy as np

In [2]:
size = 5000
k = 30
min_len = 10
max_cp = 40
pen = 100
test_series = np.hstack([np.random.normal(0, 1, (size,)),
                         np.random.normal(6, 1, (size,)),
                         np.random.normal(0, 2, (size,)),
                         np.random.normal(-4, 1, (size,)),
                         np.random.normal(3, 1, (size,)),
                         ] * 5)

In [3]:
# No jitting
class NoJitCost:
    
    def __init__(self, k):
        self.k = k
    
    def fit(self, x):
        self.n = x.shape[0]
        partial_sums = np.zeros(shape=(self.n + 1, self.k), dtype=x.dtype)
        sorted_data = np.sort(x)

        for i in np.arange(self.k):

            z = -1 + (2 * i + 1.0) / self.k
            p = 1.0 / (1 + (2 * self.n - 1) ** (-z))
            t = sorted_data[int((self.n - 1) * p)]

            for j in np.arange(1, self.n + 1):

                partial_sums[j, i] = partial_sums[j - 1, i]
                if x[j - 1] < t:
                    partial_sums[j, i] += 2
                if x[j - 1] == t:
                    partial_sums[j, i] += 1
        self.c = 2.0 * (-np.log(2 * self.n - 1))
        self.y = partial_sums
        return self
    
    def predict(self, start, end):
        n = len(end)
        all_costs = np.zeros((n,))
        ck = self.c / self.k
        for i in range(n):
            cost = 0
            s = start[i]
            e = end[i]
            ys = self.y[s, :]
            ye = self.y[e, :]
            for j in range(self.k):
                a_sum = ye[j] - ys[j]
                if a_sum != 0.0:
                    diff = e - s
                    a_half = 0.5 * a_sum
                    if a_half != diff:
                        f = a_half / diff
                        fi = 1.0 - f
                        flog = log(f)
                        filog = log(fi)
                        t = f * flog
                        ti = fi * filog
                        l = t + ti
                        ld = diff * l
                        cost += ld
            all_costs[i] = ck * cost
        return all_costs

In [4]:
def seg_fn(cost, n, min_len, max_cp, penalty):
    """Runs binary segmentation on time series"""

    # Setting up summary statistics and objects
    is_candidate = np.arange(min_len, n - min_len)
    cps = np.zeros(shape=(n,))
    costs = np.full(shape=n, fill_value=0.0)
    cps[-1] = 1
    cps[0] = 1
    costs[-1] = cost.predict(np.array([0]), np.array([n]))[0]

    # Iterating through changepoints until convergence
    while True:

        # Single Loop Iteration
        _cps = np.flatnonzero(cps)
        best_cand, best_cost, best_next_cost, best_next = 0, 0, 0, 0
        best_total_cost = costs.sum()

        # Looping over candidates
        for c1, c2 in np.stack((_cps[:-1], _cps[1:]), axis=-1):
            _cands = is_candidate[(is_candidate > c1) & (is_candidate < c2)]
            if _cands.shape[0] == 0:
                continue
            _costs = np.empty(shape=(_cands.shape[0], 3), dtype=np.float64)
            _other_costs = costs[: (c1 + 1)].sum() + costs[(c2 + 1):].sum()
            _costs[:, 0] = cost.predict(np.repeat(c1, _cands.shape[0]), _cands)
            _costs[:, 1] = cost.predict(_cands, np.repeat(c2, _cands.shape[0]))
            _costs[:, 2] = _costs[:, 0] + _costs[:, 1] + _other_costs + penalty
            _best_cand = np.argmin(_costs[:, 2])
            if _costs[_best_cand, 2] < best_total_cost:
                best_cand = _cands[_best_cand]
                best_cost = _costs[_best_cand, 0]
                best_next_cost = _costs[_best_cand, 1]
                best_total_cost = _costs[_best_cand, 2]
                best_next = c2

        if best_cand == 0:
            break
        else:
            cps[best_cand] = True
            costs[best_cand] = best_cost
            costs[best_next] = best_next_cost
            is_candidate[(best_cand - min_len): (best_cand + min_len)] = False
            if np.flatnonzero(cps).shape[0] > max_cp + 2:
                break
        
    return np.flatnonzero(cps)[1:-1]

In [5]:
class NoJitCP:

    def __init__(self, seg_fn, cost, min_len, max_cp):
        self.seg_fn = seg_fn
        self.cost = cost
        self.min_len = min_len
        self.max_cp = max_cp
    
    def fit(self, x):
        self.cost.fit(x)
        cost = self.cost
        seg_fn = self.seg_fn
        min_len = self.min_len
        max_cp = self.max_cp
        n = self.cost.n
        self.cp_fn = lambda cost, pen: seg_fn(cost, n, min_len, max_cp, pen)
        return self
    
    def predict(self, pen):
        return self.cp_fn(self.cost, pen)

In [6]:
cp0 = NoJitCP(seg_fn, NoJitCost(k), min_len, max_cp).fit(test_series)
cp0.predict(pen)

array([ 500, 1000, 1500, 2000, 2500, 3000, 3500, 4000, 4500], dtype=int64)

In [6]:
class Cost:
    
    def __init__(self, k):
        self.k = k
    
    def fit(self, x, fastmath):
        self.n = x.shape[0]
        partial_sums = np.zeros(shape=(self.n + 1, self.k), dtype=x.dtype)
        sorted_data = np.sort(x)

        for i in np.arange(self.k):

            z = -1 + (2 * i + 1.0) / self.k
            p = 1.0 / (1 + (2 * self.n - 1) ** (-z))
            t = sorted_data[int((self.n - 1) * p)]

            for j in np.arange(1, self.n + 1):

                partial_sums[j, i] = partial_sums[j - 1, i]
                if x[j - 1] < t:
                    partial_sums[j, i] += 2
                if x[j - 1] == t:
                    partial_sums[j, i] += 1
        c = 2.0 * (-np.log(2 * self.n - 1))
        self.partial_sums = partial_sums

        return self.make_cost(partial_sums, c, self.k, fastmath)
    
    @staticmethod
    def make_cost(x, c, k, fastmath):

        @nb.njit(fastmath=fastmath)
        def _cost_fn(start, end):
            n = len(end)
            all_costs = np.zeros((n,))
            ck = c / k
            for i in range(n):
                cost = 0
                s = start[i]
                e = end[i]
                ys = x[s, :]
                ye = x[e, :]
                for j in range(k):
                    a_sum = ye[j] - ys[j]
                    if a_sum != 0.0:
                        diff = e - s
                        a_half = 0.5 * a_sum
                        if a_half != diff:
                            f = a_half / diff
                            fi = 1.0 - f
                            flog = log(f)
                            filog = log(fi)
                            t = f * flog
                            ti = fi * filog
                            l = t + ti
                            ld = diff * l
                            cost += ld
                all_costs[i] = ck * cost
            return all_costs
    

        return _cost_fn

In [7]:
def seg_fn2(cost_fn, n, min_len, max_cp, penalty):
    """Runs binary segmentation on time series"""

    # Setting up summary statistics and objects
    is_candidate = np.arange(min_len, n - min_len)
    cps = np.zeros(shape=(n,))
    costs = np.full(shape=n, fill_value=0.0)
    cps[-1] = 1
    cps[0] = 1
    costs[-1] = cost_fn(np.array([0]), np.array([n]))[0]

    # Iterating through changepoints until convergence
    while True:

        # Single Loop Iteration
        _cps = np.flatnonzero(cps)
        best_cand, best_cost, best_next_cost, best_next = 0, 0, 0, 0
        best_total_cost = costs.sum()

        # Looping over candidates
        for c1, c2 in np.stack((_cps[:-1], _cps[1:]), axis=-1):
            _cands = is_candidate[(is_candidate > c1) & (is_candidate < c2)]
            if _cands.shape[0] == 0:
                continue
            _costs = np.empty(shape=(_cands.shape[0], 3), dtype=np.float64)
            _other_costs = costs[: (c1 + 1)].sum() + costs[(c2 + 1):].sum()
            _costs[:, 0] = cost_fn(np.repeat(c1, _cands.shape[0]), _cands)
            _costs[:, 1] = cost_fn(_cands, np.repeat(c2, _cands.shape[0]))
            _costs[:, 2] = _costs[:, 0] + _costs[:, 1] + _other_costs + penalty
            _best_cand = np.argmin(_costs[:, 2])
            if _costs[_best_cand, 2] < best_total_cost:
                best_cand = _cands[_best_cand]
                best_cost = _costs[_best_cand, 0]
                best_next_cost = _costs[_best_cand, 1]
                best_total_cost = _costs[_best_cand, 2]
                best_next = c2

        if best_cand == 0:
            break
        else:
            cps[best_cand] = True
            costs[best_cand] = best_cost
            costs[best_next] = best_next_cost
            is_candidate[(best_cand - min_len): (best_cand + min_len)] = False
            if np.flatnonzero(cps).shape[0] > max_cp + 2:
                break
        
    return np.flatnonzero(cps)[1:-1]

In [34]:
# class Cost:
    
#     def fit(self, x):
#         self.n = x.shape[0]
#         n = x.shape[0]
#         sum_stats = np.stack((np.append(0, x.cumsum()),
#                               np.append(0, (x ** 2).cumsum()),
#                               np.append(0, ((x - x.mean()) ** 2).cumsum()),
#                               np.arange(0, n + 1, dtype=x.dtype)
#                              ),
#                              axis=-1)

#         return self.make_cost(sum_stats)
    
#     @staticmethod
#     def make_cost(x):

#         #@nb.njit([(nb.typeof(100), nb.typeof(100),)], fastmath=True, parallel=True, nogil=True)
#         #@nb.njit(fastmath=True, parallel=True, nogil=True)
#         def _cost_fn(start, end):
#             _x = x[end, :] - x[start, :]
#             return _x[:, 3] * (np.log(2 * np.pi) + np.log(np.fmax((_x[:, 1] - ((_x[:, 0] * _x[:, 0]) / _x[:, 3]))/ _x[:, 3], x.dtype.type(1e-8)) + 1))

#         return _cost_fn

In [8]:
class CP:
    
    def __init__(self, seg_fn, cost, min_len, max_cp):
        self.seg_fn = seg_fn
        self.cost = cost
        self.min_len = min_len
        self.max_cp = max_cp
    
    def fit(self, x, fastmath):
        cost_fn = self.cost.fit(x, fastmath)
        seg_fn = self.seg_fn
        min_len = self.min_len
        max_cp = self.max_cp
        n = self.cost.n
        return_type = nb.typeof(np.argmin([100]))
        nb_seg_fn = nb.njit(seg_fn, fastmath=fastmath)
        #nb_seg_fn.compile((nb.typeof(cost_fn), nb.typeof(n), nb.typeof(min_len), nb.typeof(max_cp), nb.typeof(100)))
        self.cp_fn = nb.njit(lambda pen: nb_seg_fn(cost_fn, n, min_len, max_cp, pen))
        #self.cp_fn.compile((nb.typeof(100),))
        #self.cp_fn = lambda pen: seg_fn(cost_fn, n, min_len, max_cp, pen)
        return self
    
    def predict(self, pen):
        return self.cp_fn(pen)

In [9]:
cp = CP(seg_fn2, Cost(k), min_len, max_cp).fit(test_series, True)
cp.predict(pen)

array([  5000,  10000,  15000,  20000,  25000,  30000,  35000,  40000,
        45000,  50000,  55000,  60000,  65000,  70000,  75000,  80000,
        85000,  90000,  95000, 100000, 105000, 110002, 115000, 120000],
      dtype=int64)

In [10]:
# Replacing with jitclass
spec = [
    ('k', nb.typeof(10)),
    ('n', nb.typeof(test_series.shape[0])),
    ('y', nb.typeof(np.zeros(shape=(1, 1), dtype=test_series.dtype))),
    ('c', nb.typeof(2.0 * (-np.log(2 * test_series.shape[0] - 1))))
]

@nb.experimental.jitclass(spec)
class Cost2:
    
    def __init__(self, k):
        self.k = k
    
    def fit(self, x):
        self.n = x.shape[0]
        partial_sums = np.zeros(shape=(self.n + 1, self.k), dtype=x.dtype)
        sorted_data = np.sort(x)

        for i in np.arange(self.k):

            z = -1 + (2 * i + 1.0) / self.k
            p = 1.0 / (1 + (2 * self.n - 1) ** (-z))
            t = sorted_data[int((self.n - 1) * p)]

            for j in np.arange(1, self.n + 1):

                partial_sums[j, i] = partial_sums[j - 1, i]
                if x[j - 1] < t:
                    partial_sums[j, i] += 2
                if x[j - 1] == t:
                    partial_sums[j, i] += 1
        self.c = 2.0 * (-np.log(2 * self.n - 1))
        self.y = partial_sums
    
    def predict(self, start, end):
        n = len(end)
        all_costs = np.zeros((n,))
        ck = self.c / self.k
        for i in range(n):
            cost = 0
            s = start[i]
            e = end[i]
            ys = self.y[s, :]
            ye = self.y[e, :]
            for j in range(self.k):
                a_sum = ye[j] - ys[j]
                if a_sum != 0.0:
                    diff = e - s
                    a_half = 0.5 * a_sum
                    if a_half != diff:
                        f = a_half / diff
                        fi = 1.0 - f
                        flog = log(f)
                        filog = log(fi)
                        t = f * flog
                        ti = fi * filog
                        l = t + ti
                        ld = diff * l
                        cost += ld
            all_costs[i] = ck * cost
        return all_costs

In [11]:
class CP2:

    def __init__(self, seg_fn, cost, min_len, max_cp):
        self.seg_fn = seg_fn
        self.cost = cost
        self.min_len = min_len
        self.max_cp = max_cp
    
    def fit(self, x):
        self.cost.fit(x)
        cost = self.cost
        seg_fn = self.seg_fn
        min_len = self.min_len
        max_cp = self.max_cp
        n = self.cost.n
        
        nb_seg_fn = nb.njit(seg_fn)#, fastmath=True)
        #nb_seg_fn.compile((nb.typeof(cost), nb.typeof(n), nb.typeof(min_len), nb.typeof(max_cp), nb.typeof(100)))
        self.cp_fn = nb.njit(lambda cost, pen: nb_seg_fn(cost, n, min_len, max_cp, pen))
        #self.cp_fn.compile((nb.typeof(cost), nb.typeof(100)))
        
        return self
    
    def predict(self, pen):
        return self.cp_fn(self.cost, pen)

In [12]:
cp2 = CP2(seg_fn, Cost2(k), min_len, max_cp).fit(test_series)
cp2.predict(pen)

array([  5000,  10000,  15000,  20000,  25000,  30000,  35000,  40000,
        45000,  50000,  55000,  60000,  65000,  70000,  75000,  80000,
        85000,  90000,  95000, 100000, 105000, 110002, 115000, 120000],
      dtype=int64)

In [16]:
Cost3Type = nb.deferred_type()

spec = [
    ('k', nb.typeof(10)),
    ('n', nb.typeof(test_series.shape[0])),
    ('y', nb.typeof(np.zeros(shape=(1, 1), dtype=test_series.dtype))),
    ('c', nb.typeof(2.0 * (-np.log(2 * test_series.shape[0] - 1))))
]

@nb.experimental.jitclass(spec)
class Cost3:
    
    def __init__(self, k):
        self.k = k
    
    def fit(self, x):
        self.n = x.shape[0]
        partial_sums = np.zeros(shape=(self.n + 1, self.k), dtype=x.dtype)
        sorted_data = np.sort(x)

        for i in np.arange(self.k):

            z = -1 + (2 * i + 1.0) / self.k
            p = 1.0 / (1 + (2 * self.n - 1) ** (-z))
            t = sorted_data[int((self.n - 1) * p)]

            for j in np.arange(1, self.n + 1):

                partial_sums[j, i] = partial_sums[j - 1, i]
                if x[j - 1] < t:
                    partial_sums[j, i] += 2
                if x[j - 1] == t:
                    partial_sums[j, i] += 1
        self.c = 2.0 * (-np.log(2 * self.n - 1))
        self.y = partial_sums
        return self

Cost3Type.define(Cost3.class_type.instance_type)

@nb.extending.overload_method(nb.types.misc.ClassInstanceType, 'predict', jit_options={'cache': False, 'fastmath': True, 'parallel': False, 'nogil': False, 'boundscheck': False, 'inline': 'always'})
def cost_fast(inst, start, end):
    if inst is Cost3.class_type.instance_type:

        def impl(inst, start, end):
            n = len(end)
            all_costs = np.zeros((n,))
            ck = inst.c / inst.k
            for i in range(n):
                cost = 0
                s = start[i]
                e = end[i]
                ys = inst.y[s, :]
                ye = inst.y[e, :]
                for j in range(inst.k):
                    a_sum = ye[j] - ys[j]
                    if a_sum != 0.0:
                        diff = e - s
                        a_half = 0.5 * a_sum
                        if a_half != diff:
                            f = a_half / diff
                            fi = 1.0 - f
                            flog = log(f)
                            filog = log(fi)
                            t = f * flog
                            ti = fi * filog
                            l = t + ti
                            ld = diff * l
                            cost += ld
                all_costs[i] = ck * cost
            return all_costs

        return impl

In [18]:
cp3 = CP2(seg_fn, Cost3(k), min_len, max_cp).fit(test_series)
cp3.predict(pen)

array([  5000,  10000,  15000,  20000,  25000,  30000,  35000,  40000,
        45000,  50000,  55000,  60000,  65000,  70000,  75000,  80000,
        85000,  90000,  95000, 100000, 105000, 110002, 115000, 120000],
      dtype=int64)

In [15]:
# No jitting
%timeit cp0.predict(pen)

7.76 s ± 374 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [14]:
# Function factory
# With flags
%timeit cp.predict(pen)

1.56 s ± 54.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:
# Jitclass
%timeit cp2.predict(pen)

1.85 s ± 135 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [19]:
# Jitclass with overloading
# With flags
%timeit cp3.predict(pen)
# Overloading seems to win!

1.61 s ± 85.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
# Testing against changepoint.np
from pychange.r import RChangepoint
R = RChangepoint('np', penalty='Manual', pen_value=pen, nquantiles=k).fit(test_series)
R.predict()

In [27]:
%load_ext line_profiler

In [22]:
c0 = NoJitCost(k).fit(test_series)
#c1 = Cost(k).fit(test_series)
#c2 = Cost2(k).fit(test_series)
#c3 = cost3(k).fit(test_series)

In [220]:
from math import log
def predict(y, start, end, k, c):
    n = len(end)
    all_costs = np.zeros((n,))
    ck = c / k
    for i in range(n):
        cost = 0
        s = start[i]
        e = end[i]
        ys = y[s, :]
        ye = y[e, :]
        for j in range(k):
            a_sum = ye[j] - ys[j]
            if a_sum != 0.0:
                diff = e - s
                a_half = 0.5 * a_sum
                if a_half != diff:
                    f = a_half / diff
                    fi = 1.0 - f
                    flog = log(f)
                    filog = log(fi)
                    t = f * flog
                    ti = fi * filog
                    l = t + ti
                    ld = diff * l
                    cost += ld
        all_costs[i] = ck * cost
    return all_costs

predict_nb = nb.njit(predict)
predict_nb_fast = nb.njit(predict, fastmath=True, nogil=True)
_ = predict_nb(c0.y, np.arange(50), np.arange(c0.n - 50, c0.n), k, c0.c)
_ = predict_nb_fast(c0.y, np.arange(50), np.arange(c0.n - 50, c0.n), k, c0.c)

In [210]:
x = test_series
n = test_series.shape[0]
sum_stats = np.stack((np.append(0, x.cumsum()),
                              np.append(0, (x ** 2).cumsum()),
                              np.append(0, ((x - x.mean()) ** 2).cumsum()),
                              np.arange(0, n + 1, dtype=x.dtype)
                             ),
                             axis=-1)

In [211]:
log(2.0 * np.pi)

1.8378770664093453

In [291]:
from math import log
def predict(y, start, end):
    n = len(start)
    costs = np.zeros((n,))
    for i in range(n):
        d = y[end[i]] - y[start[i]]
        a1 = d[0] ** 2 / d[3]
        a2 = d[0] - a1
        a3 = a2 / d[3]
        if a3 <= 0.0:
            a3 = 1e-6
        a4 = d[3] * 1.8378771 + log(a3 + 1.0)
        costs[i] = a4
    return costs

#predict_nb = nb.njit(predict)
predict_nb_fast = nb.njit(predict, fastmath=True)
#_ = predict_nb(sum_stats, np.arange(50), np.arange(n - 50, n))
_ = predict_nb_fast(sum_stats, np.arange(50), np.arange(n - 50, n))

In [292]:
%timeit predict_nb_fast(sum_stats, np.arange(50), np.arange(n - 50, n))

6.88 µs ± 136 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [288]:
%timeit predict_nb_fast(sum_stats, np.arange(50), np.arange(n - 50, n))

7.05 µs ± 212 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [226]:
%timeit predict(sum_stats, np.arange(50), np.arange(n - 50, n))

113 µs ± 1.35 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [227]:
%timeit predict_nb(sum_stats, np.arange(50), np.arange(n - 50, n))

7.43 µs ± 148 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [238]:
%timeit predict_nb_fast(sum_stats, np.arange(50), np.arange(n - 50, n))

7.27 µs ± 111 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [222]:
%timeit predict(c0.y, np.arange(50), np.arange(c0.n - 50, c0.n), k, c0.c)

4.72 ms ± 61.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [223]:
%timeit predict_nb(c0.y, np.arange(50), np.arange(c0.n - 50, c0.n), k, c0.c)

25.8 µs ± 544 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [224]:
%timeit predict_nb_fast(c0.y, np.arange(50), np.arange(c0.n - 50, c0.n), k, c0.c)

11.3 µs ± 193 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [218]:
%timeit predict_nb_fast(c0.y, np.arange(50), np.arange(c0.n - 50, c0.n), k, c0.c)

70.3 µs ± 481 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [219]:
predict_nb_fast.parallel_diagnostics(level=4)

 
 Parallel Accelerator Optimizing:  Function predict, <ipython-
input-217-bacf83a4ae56> (2)  


Parallel loop listing for  Function predict, <ipython-input-217-bacf83a4ae56> (2) 
----------------------------------------------------|loop #ID
def predict(y, start, end, k, c):                   | 
    n = len(end)                                    | 
    all_costs = np.zeros((n,))----------------------| #15
    ck = c / k                                      | 
    for i, (s, e) in enumerate(zip(start, end)):    | 
        cost = 0                                    | 
        ys = y[s, :]                                | 
        ye = y[e, :]                                | 
        for j in range(k):                          | 
            a_sum = ye[j] - ys[j]                   | 
            if a_sum != 0.0:                        | 
                diff = e - s                        | 
                a_half = 0.5 * a_sum                | 
                if a_half != diff:      

In [221]:
%lprun -f predict predict(c0.y, np.array([0]), np.array([c0.n]), k, c0.c)

In [280]:
%lprun -f predict predict(sum_stats, np.arange(50), np.arange(n - 50, n))

In [22]:
%lprun c0.predict([0], [test_series.shape[0]])

AttributeError: 'function' object has no attribute 'predict'